# Natural Langauge Processing - Assignment 02

### Importing of Natural Language Toolkit

In [148]:
import nltk
from nltk.util import ngrams
from nltk.tokenize import sent_tokenize as st, word_tokenize as wt
import pandas as pd
import numpy as np
from operator import itemgetter
import random
import string
import math
import re
from collections import Counter

### Q1) Parsing of dataset (Alice in Wonderland)

In [149]:
text = (open('holmes.txt','r')).read()

### Variables initialization

In [150]:
tokens = 0
types_set = set()
list_all_words = []
sent_token_list = []
unigrams = {} # dictionary for maintaining count of unigrams
MLE_Unigrams = {} # dictionary for maintaining MLE for Unigrams
MLE_Bigrams = {} # dictionary for maintaining MLE for Bigrams
MLE_Trigrams = {} # dictionary for maintaining MLE for Trigrams
MLE_Quadgrams = {} # dictionary for maintaining MLE for Quadgrams
Add_1_Bigrams = {} # dictionary for maintaining Add-one estimation of Bigrams
eff_bigrams = {} # dictionary for maintaining Effective Bigram counts as effect of Add-one smoothing
p = r'[?|$|&|*|%|@|(|)|~|.|:|,|;|\'|,|`|!|\--]'

### Q2) Sentence Tokenization and filtering out unnecessary symbols, newlines, etc.

In [151]:
sent_tokenize_list = st(text)
total_sents = len(sent_tokenize_list)
pattern = r'[^a-zA-Z0-9]'
for i in range(total_sents):
    sentence = sent_tokenize_list[i].strip()
    sent_tokenize_list[i] = re.sub(pattern, r' ', sentence)
    temp = sent_tokenize_list[i].split()
    sent_tokenize_list[i] = " ".join(temp) # remove whitespaces
    sent_token_list.append(sent_tokenize_list[i])
    sent_tokenize_list[i] = "<s> "+ " ".join(temp) + " </s>"
sent_tokenize_list = [x.lower() for x in sent_tokenize_list]
sent_token_list = [x.lower() for x in sent_token_list]
#sent_token_list

### For whole Corpus:
t = 0
types = set()
unigram = {}
for i in range(total_sents):
    t_in_line = wt(sent_token_list[i])
    t = t + len(t_in_line)
    types.update(t_in_line)      
for word in types:
    unigram[word] = 0
for i in range(total_sents):
    token_in_line = wt(sent_token_list[i]);
    for word in token_in_line:
        unigram[word] += 1
unigram['<s>'] = 6820
unigram['<\s>'] = 6820
len(unigram)

7887

### Splitting of Training and test data:

In [152]:
train_data_size = int(total_sents*0.8)
train_data = random.sample(sent_token_list, train_data_size)
test_data = [n for n in sent_token_list if n not in train_data]
test_data_size = len(test_data)

### Q3) MLE for unigrams (on training corpus):

In [153]:
for i in range(train_data_size):
    token_in_line = wt(train_data[i])
    tokens = tokens + len(token_in_line)
    types_set.update(token_in_line)      #updating the list of types line by line i.e. unique words
V = len(types_set)
for word in types_set:
    unigrams[word] = 0
for i in range(train_data_size):
    token_in_line = wt(train_data[i]);
    for word in token_in_line:
        unigrams[word] += 1
unigrams['<s>'] = 6820
unigrams['<\s>'] = 6820

for word in unigrams.keys():
    freq = unigrams[word]
    MLE_Unigrams[word] = (freq/tokens)*100   # multiplied with 100 just to show the probabalities in more readable format
len(unigrams)

7235

### MLE for Bigrams(on whole Corpus):

In [154]:
bigram = Counter()
MLE_Bigram = {}
for sentnc in sent_token_list:
    token = wt(sentnc)
    b = Counter(ngrams(token, 2, pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='</s>'))
    dict.update(bigram, b)
for k,v in bigram.items():
    key = k[0]
    if key in unigram.keys():
        MLE_Bigram[k] = v/unigram[key]
    else:
        MLE_Bigram[k] = 0
#print(MLE_Bigram)

### MLE for Bigrams(on Training corpus.):

In [155]:
bigrams = Counter()
for sentnc in train_data:
    token = wt(sentnc)
    b = Counter(ngrams(token, 2, pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='</s>'))
    dict.update(bigrams, b)
for k,v in bigrams.items():
    key = k[0]
    if key in unigrams.keys():
        MLE_Bigrams[k] = v/unigrams[key]
    else:
        MLE_Bigrams[k] = 0
len(bigrams)

42219

### MLE for Trigrams (on Training corpus.):

In [120]:
trigrams = Counter()
for sentnc in train_data:
    token = wt(sentnc)
    t = Counter(ngrams(token, 3, pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='</s>'))
    dict.update(trigrams, t)
#print(trigrams)
for k,v in trigrams.items():
    key = (k[0],k[1])
    if key in dict(bigrams).keys():
        MLE_Trigrams[k] = v/bigrams[key]
    else:
        MLE_Trigrams[k] = 0
#MLE_Trigrams

### MLE for Quadgrams(on Training corpus.):

In [373]:
quadgrams = Counter()
for sentnc in train_data:
    token = wt(sentnc)
    q = Counter(ngrams(token, 4, pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='</s>'))
    dict.update(quadgrams, q)
#print(quadgrams)
for k,v in quadgrams.items():
    key = (k[0],k[1],k[2])
    if key in dict(trigrams).keys():
        MLE_Quadgrams[k] = v/trigrams[key]
    else:
        MLE_Quadgrams[k] = 0
#MLE_Quadgrams

### How many n-grams are possible and how many actually exists? 

In [156]:
V = len(unigram)
print(V," unigrams are possible and ",V," actually exists")
print(len(bigram)," bigrams are possible and ",V*V," actually exists")

7887  unigrams are possible and  7887  actually exists
49220  bigrams are possible and  62204769  actually exists


### Q4 a) Developing a Generator: 

In [157]:
def gen(model):
    sentence = []
    p_list = []
    if model=='bigram':
        word = '<s>'
        sentence.append(word)
        t= '</s>'
        for i in range(8):
            for k,v in MLE_Bigrams.items():
                key = k[0]
                if key==word:
                    p_list.append(k[1])
            new_word = random.choice(p_list)
            sentence.append(new_word)
            t = new_word   #### until it founds last word </s>
        print(sentence)
        
gen('bigram')

['<s>', 'swiftly', 'mrs', 'far', 'though', 'remarkable', 'dear', 'think', 'in']


### Q4 b) Computing the probability of a given sentence in log-space:

In [158]:
def prob(sentence,model_name):
    log_p = 1
    sentence = sentence.lower();
    token = wt(sentence)
    b = list(ngrams(token, 2, pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='</s>'))
    for key in b:
        log_p = MLE_Bigram[key]
        log_p += math.log10(log_p)
    print(log_p)
    
prob("I have seldom heard him mention her under any other name",MLE_Bigram)

-1.7762626572401894


### Q5) Add-1 Smoothing for Bigram model:

In [159]:
for k,v in bigrams.items():
    key = k[0]
    if key in unigrams.keys():
        Add_1_Bigrams[k] = (v+1)/(unigrams[key]+V)
        eff_bigrams[k] = Add_1_Bigrams[k]*unigrams[key]
    else:
        Add_1_Bigrams[k] = 0
        eff_bigrams[k] = 0
#print(Add_1_Bigrams)
#print(eff_bigrams)
print("Bigram count of ('gibe','and'):",bigrams['gibe','and'],"  and  Effective Bigram count,c* of ('gibe','and'):",eff_bigrams['gibe','and'])
print("Bigram count of ('senility','</s>'):",bigrams['senility','</s>'],"  and  Effective Bigram count,c* of ('senility','</s>'):",eff_bigrams['senility','</s>'])


Bigram count of ('gibe','and'): 1   and  Effective Bigram count,c* of ('gibe','and'): 0.0002535496957403651
Bigram count of ('senility','</s>'): 1   and  Effective Bigram count,c* of ('senility','</s>'): 0.0002535496957403651


This Drastic change occurs in the counts(post-smoothing) because in Add-one smoothing too much weight is given to unseen ngrams

### Q6) Good-turing smoothing technique:

In [78]:
c = [0,1,2,3,4,5,6,7,8,9]
good_c = []
N = {1:0,2:0,3:0,4:0,5:0,6:0,7:0,8:0,9:0,10:0}
#uni_count = 0
for word,freq in unigrams.items():
    if freq >= 1 and freq <= 10:
        N[freq] += 1
        #uni_count += 1
for i in c:
    if i == 0:
        z = N[1]/len(unigrams)
    else:
        z = ((i+1)*N[i+1])/N[i]
    good_c.append(z)
d = []
for i in range(len(c)):
    d.append(c[i]-good_c[i])
print("d: ",sum(d) / len(d) )

d:  0.3832345362692761


### Q7) Computation of the perplexity value for the test dataset for the bigram model using add-1:

In [87]:
PP1 = 1 # perplexity
Add_1_test_Bigrams = {}
x = 0
types_test = set()
test_unigrams = {}

for i in range(test_data_size):
    t_in_line = wt(test_data[i])
    x = x + len(t_in_line)
    types_test.update(t_in_line)  
V_test = len(types_test)
for word in types_test:
    test_unigrams[word] = 0
for i in range(test_data_size):
    token_in_line = wt(test_data[i]);
    for word in token_in_line:
        test_unigrams[word] += 1
test_unigrams['<s>'] = 6820
test_unigrams['<\s>'] = 6820

test_bigrams = Counter()
for sentnc in test_data:
    token = wt(sentnc)
    b = Counter(ngrams(token, 2, pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='</s>'))
    dict.update(test_bigrams, b)
n = len(test_bigrams)

for k,v in test_bigrams.items():
    key = k[0]
    if key in test_unigrams.keys():
        Add_1_test_Bigrams[k] = (v+1)/(test_unigrams[key]+V_test)
    else:
        Add_1_test_Bigrams[k] = 0

for k,v in Add_1_test_Bigrams.items():
    p = (1/v)**(1/n)
    PP1 *= p
print("Perplexity on Test data: ",PP1)

Perplexity on Test data:  1816.5767843481103


### Computation of the perplexity value for the test dataset for the bigram model using Good-turing:

In [88]:
PP2 = 1 # perplexity
good_turing_test_Bigrams = []
n_test = 0
c_test = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
good_c_test = []
N_test = {1:0,2:0,3:0,4:0,5:0,6:0,7:0,8:0,9:0,10:0,11:0,12:0,13:0,14:0,15:0,16:0,17:0,18:0,19:0,20:0}
for word,freq in test_unigrams.items():
    if freq >= 1 and freq <= 20:
        N_test[freq] += 1
        n_test += 1
for i in c_test:
    if i == 0:
        z = N_test[1]/n_test
    else:
        z = ((i+1)*N_test[i+1])/N_test[i]
    good_c_test.append(z)
        
for i in good_c_test:
    good_turing_test_Bigrams.append(i/n_test)
    
for i in good_turing_test_Bigrams:
    p = (1/i)**(-1/n_test)
    PP2 *= p
print("Perplexity on Test data: ",PP2)

Perplexity on Test data:  0.9626914512342537


Good Turing performs better as it has low perplexity value.